In [1]:
import pandas as pd

df3 = pd.read_csv(r"/Users/johaunh/Downloads/df3.csv")
df3.head()

,DATE_FDA_FIRST_RECEIVED_REPORT,REPORT_ID,DATE_EVENT,PRODUCT_TYPE,PRODUCT,PRODUCT_CODE,DESCRIPTION,SEX,CASE_MEDDRA_PREFERRED_TERMS,CASE_OUTCOME,age_in_years
0,6/30/2023,2023-CFS-006722,6/27/2023,SUSPECT,AIRLINE FOOD SERVED BY UNITED AIRLINES,37,Mult Food Dinner/Grav/Sauce/Special,Male,"Diarrhoea, Food poisoning, Pyrexia, Tremor, Vo...",Other Serious or Important Medical Event,45.0
1,6/30/2023,2023-CFS-006717,NaN,SUSPECT,METAMUCIL,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Pain, Vomiting","Hospitalization, Other Serious or Important Me...",46.0
2,6/30/2023,2023-CFS-006712,6/25/2023,SUSPECT,COKE ZERO,29,Soft Drink/Water,Male,"Disorientation, Fatigue, Seizure","Life Threatening, Hospitalization, Other Serio...",65.0
3,6/29/2023,2023-CFS-006687,6/20/2023,SUSPECT,HUEL BLACK EDITION,37,Mult Food Dinner/Grav/Sauce/Special,Male,Food poisoning,Other Serious or Important Medical Event,39.0
4,6/29/2023,2023-CFS-006682,6/27/2023,SUSPECT,GINKGO BILOBA,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Cardiac failure, Death, Hypoglycaemia, Renal f...","Death, Hospitalization",92.0


In [2]:
terms = df3["CASE_MEDDRA_PREFERRED_TERMS"].str.lower().to_list()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize medical terms
vectorizer_c = CountVectorizer(tokenizer=lambda x: x.split(", "))
count_matrix = vectorizer_c.fit_transform(terms)

/Users/johaunh/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
count_df = pd.DataFrame(count_matrix.toarray(), columns=vectorizer_c.get_feature_names_out())
count_df.head()

,abasia,abdominal abscess,abdominal adhesions,abdominal compartment syndrome,abdominal discomfort,abdominal distension,abdominal hernia,abdominal hernia obstructive,abdominal infection,abdominal injury,...,x-ray,x-ray abnormal,x-ray gastrointestinal tract abnormal,x-ray of pelvis and hip abnormal,x-ray with contrast upper gastrointestinal tract,xanthopsia,yawning,yellow skin,yersinia infection,zoonosis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

symptom_embeddings = model.encode(count_df.columns)

In [6]:
terms_series = count_df.sum().sort_values(ascending=False)
terms_series

death                        18240
ovarian cancer               15466
diarrhoea                    11466
injury                       11416
nausea                       10224
                             ...  
lip ulceration                   1
lipase decreased                 1
lipid metabolism disorder        1
lipids                           1
gravitational oedema             1
Length: 4515, dtype: int64

In [7]:
def create_medical_embeddings(row):
    # 
    return list(symptom_embeddings[row.astype(bool)].mean(axis=0))

In [8]:
df3["pref_med_emb"] = count_df.apply(create_medical_embeddings, axis=1)

In [9]:
df3.head()

,DATE_FDA_FIRST_RECEIVED_REPORT,REPORT_ID,DATE_EVENT,PRODUCT_TYPE,PRODUCT,PRODUCT_CODE,DESCRIPTION,SEX,CASE_MEDDRA_PREFERRED_TERMS,CASE_OUTCOME,age_in_years,pref_med_emb
0,6/30/2023,2023-CFS-006722,6/27/2023,SUSPECT,AIRLINE FOOD SERVED BY UNITED AIRLINES,37,Mult Food Dinner/Grav/Sauce/Special,Male,"Diarrhoea, Food poisoning, Pyrexia, Tremor, Vo...",Other Serious or Important Medical Event,45.0,"[0.018172732, -0.031314567, 0.026723817, 0.022..."
1,6/30/2023,2023-CFS-006717,NaN,SUSPECT,METAMUCIL,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Pain, Vomiting","Hospitalization, Other Serious or Important Me...",46.0,"[-0.004000198, -0.008378927, 0.017452758, 0.06..."
2,6/30/2023,2023-CFS-006712,6/25/2023,SUSPECT,COKE ZERO,29,Soft Drink/Water,Male,"Disorientation, Fatigue, Seizure","Life Threatening, Hospitalization, Other Serio...",65.0,"[-0.012664515, 0.006485447, 0.030927574, 0.067..."
3,6/29/2023,2023-CFS-006687,6/20/2023,SUSPECT,HUEL BLACK EDITION,37,Mult Food Dinner/Grav/Sauce/Special,Male,Food poisoning,Other Serious or Important Medical Event,39.0,"[0.06859944, -0.049020994, 0.02800111, 0.04540..."
4,6/29/2023,2023-CFS-006682,6/27/2023,SUSPECT,GINKGO BILOBA,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Cardiac failure, Death, Hypoglycaemia, Renal f...","Death, Hospitalization",92.0,"[-0.0070610754, 0.054655064, -0.014201546, 0.0..."


In [13]:
import numpy as np

med_emb = np.array(df3["pref_med_emb"].to_list())

In [14]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=7)
kmeans.fit(med_emb)

/Users/johaunh/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7)

In [16]:
df3["pref_med_cluster"] = kmeans.labels_

In [124]:
df3["pref_med_cluster"].value_counts()

pref_med_cluster
6    48817
3    21227
0    17709
1    15602
2    12472
4     9734
5     4841
Name: count, dtype: int64

In [29]:
df3[df3["pref_med_cluster"] == 6]

,DATE_FDA_FIRST_RECEIVED_REPORT,REPORT_ID,DATE_EVENT,PRODUCT_TYPE,PRODUCT,PRODUCT_CODE,DESCRIPTION,SEX,CASE_MEDDRA_PREFERRED_TERMS,CASE_OUTCOME,age_in_years,pref_med_emb,pref_med_cluster
2,6/30/2023,2023-CFS-006712,6/25/2023,SUSPECT,COKE ZERO,29,Soft Drink/Water,Male,"Disorientation, Fatigue, Seizure","Life Threatening, Hospitalization, Other Serio...",65.0,"[-0.012664515, 0.006485447, 0.030927574, 0.067...",6
4,6/29/2023,2023-CFS-006682,6/27/2023,SUSPECT,GINKGO BILOBA,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Cardiac failure, Death, Hypoglycaemia, Renal f...","Death, Hospitalization",92.0,"[-0.0070610754, 0.054655064, -0.014201546, 0.0...",6
7,6/29/2023,2023-CFS-006666,6/26/2023,SUSPECT,DELTA-9 HIGH RISE GUMMY,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Deafness, Diplopia, Speech disorder",Other Serious or Important Medical Event,56.0,"[-0.005230278, -0.019725824, 0.019745851, 0.02...",6
16,6/28/2023,2023-CFS-006634,4/22/2023,SUSPECT,ADRENAL COCKTAIL,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Nephrolithiasis, Pain","Hospitalization, Other Serious or Important Me...",34.0,"[-0.01445077, 0.02408435, 0.036949042, 0.03184...",6
17,6/28/2023,2023-CFS-006634,4/22/2023,CONCOMITANT,Prenatal Vitamin,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"Nephrolithiasis, Pain","Hospitalization, Other Serious or Important Me...",34.0,"[-0.01445077, 0.02408435, 0.036949042, 0.03184...",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130395,3/14/2003,71647,3/13/2003,SUSPECT,PLANTERS CASHEW HALVES AND PIECES,23,Nuts/Edible Seed,Female,CHEST PAIN,Other Outcome,45.0,"[0.039125077, 0.090080366, -0.0034510428, 0.08...",6
130397,2/26/2003,79988,2/17/2003,SUSPECT,JUST FOR MEN SHAMPOO-IN HAIR COLOR,53,Cosmetics,Male,"ASTHENIA, BURNING SENSATION, EYE PAIN, SINUS PAIN",Other Outcome,55.0,"[-0.019370254, 0.0049461285, -0.009432819, 0.0...",6
130398,2/13/2003,73153,1/23/2003,SUSPECT,TRADER JOE'S CREAM OF SALMON SOUP,38,Soup,Female,"CHEST PAIN, DIZZINESS, DYSPNOEA, HYPOAESTHESIA...","Required Intervention, Visited Emergency Room",45.0,"[-0.004503767, 0.011193583, -0.004350821, 0.03...",6
130400,6/26/2002,80767,5/24/2002,SUSPECT,XENADRINE RFA-1,54,Vit/Min/Prot/Unconv Diet(Human/Animal),Female,"CORONARY ARTERY DISSECTION, CORONARY ARTERY EM...","Life Threatening, Hospitalization",50.0,"[0.002589471, 0.07845415, -0.0057395673, 0.024...",6


In [35]:
# stats

columns = ["PRODUCT_TYPE", "PRODUCT_CODE", "SEX"]

for item in columns:
    print(df3[df3["pref_med_cluster"] == 6].value_counts([item]))
    print()

df3[df3["pref_med_cluster"] == 6]["age_in_years"].describe()

PRODUCT_TYPE
SUSPECT         33926
CONCOMITANT     14891
Name: count, dtype: int64

PRODUCT_CODE
54              39314
53               4111
41                919
29                555
23                533
33                379
3                 376
16                270
31                256
5                 238
24                238
37                169
7                 166
34                158
20                148
9                 114
41G               105
13                 92
28                 88
36                 86
2                  57
14                 48
12                 42
38                 40
30                 36
27                 34
39                 33
17                 32
4                  31
26                 29
35                 21
32                 20
46                 19
18                 19
50                 15
15                 11
40                  9
52                  6
Name: count, dtype: int64

SEX   
Female    31827
Male      16990
N

count    48817.000000
mean        54.790528
std         19.960578
min          5.100000
25%         39.000000
50%         56.000000
75%         71.000000
max        100.000000
Name: age_in_years, dtype: float64

label meanings

- 0 - ovarian/uterine cancer
- 1 - hair/skin problems
- 2 - death
- 3 - abdominal/gastrointestinal issues
- 4 - cancer? includes uterine + others
- 5 - choking/throat irritation
- 6 - other (i don't know enough about medical stuff to know)

In [223]:
temp = df3[df3["pref_med_cluster"] == 5]["PRODUCT"]
temp

102                 TARGET MINS CALCIUM MAGNESIUM POTASSIUM
163                                        CITRACAL MAXIMUM
164             ONE A DAY WOMEN'S 50 PLUS HEALTHY ADVANTAGE
229                          CENTRUM SILVER WOMEN'S 50 PLUS
278                                    PRESERVISION AREDS 2
                                ...                        
129455    HI C GREEN APPLE SOUR BLAST WITH CRYSTALS IN T...
130172                                            EPHEDRINE
130288                    FLORIDA NATURAL 100% ORANGE JUICE
130354          FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS
130361                                PEPSI COLA SOFT DRINK
Name: PRODUCT, Length: 4841, dtype: object

In [218]:
# get counts for a product

sum(num for item, num in temp.items() if "peanut butter" in item)

134

# 0

- exemption 4 (17558)
- johnsons baby powder (97)
- shower to shower (35)
- puritans pride (vitamin brand) (13)

# 1

- vitamins (2564+)
- wen (hair product brand) (1798)
- devacurl (hair product brand) (1291)
- monat (hair product brand) (846)

# 2

- exemption 4 (11750)
- kratom (134)
- vitamins (78+)

# 3

- jif (peanut butter brand) (1171)
- peter pan (peanut butter brand) (191)
- peanut butter (product) (1709)
- centrum (vitamin brand) (339)

# 4

- johnson's baby powder (79)
- puritan's pride (vitamin brand) (160)
- centrum (vitamin brand) (55)
- vitamins (240+)

# 5

- one a day (vitamin brand) (552)
- centrum (vitamin brand) (1363)
- vitamins (2426+)

# 6

- preservision (vitamin brand) (975)
- centrum (vitamin brand) (1278)
- nature made (vitamin brand) (478)
- vitamins (8746)